In [1]:
import os

In [2]:
import yaml
import pandas as pd 
from box import ConfigBox

In [8]:
data = pd.read_csv("artifacts/winequality-red.csv")
data.head()

'c:\\Users\\user\\Desktop\\End-to end- ML-project-with-ML-flow\\End-to-end-Machine-Learning-Project-with-Mlflow'

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [13]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [14]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir:Path
    all_schema: dict

In [16]:
from src.Mlproject.constants import *
from  src.Mlproject.utils.common import read_yaml, create_directories

In [17]:

import logging
from pathlib import Path
from box import ConfigBox
from src.Mlproject.utils.common import read_yaml, create_directories


logging.basicConfig(level=logging.INFO, format='[%(asctime)s]: %(message)s:')

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = ConfigBox(read_yaml(config_filepath))
        self.params = ConfigBox(read_yaml(params_filepath))
        self.schema = ConfigBox(read_yaml(schema_filepath))

        logging.info(f"Loaded schema file content: {self.schema}")

        self.artifacts_root = Path(self.config.get('artifacts_root', 'artifacts'))
        create_directories([self.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.get('data_validation', {})
        schema = self.schema.get('Columns', {})
        logging.info(f"Loaded schema: {schema}")
        
        # Use default values if keys are missing
        root_dir = Path(config.get('root_dir', self.artifacts_root / 'data_validation'))
        status_file = config.get('STATUS_FILE', 'status.txt')
        unzip_data_dir = Path(config.get('unzip_data_dir', self.artifacts_root / 'data'))

        create_directories([root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=root_dir,
            STATUS_FILE=status_file,
            unzip_data_dir=unzip_data_dir,
            all_schema=schema,
        )

        logging.info(f"Data Validation Config: {data_validation_config}")
        return data_validation_config   

In [18]:
import yaml
from pathlib import Path
from box import ConfigBox
import logging

def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns

    Args:
        path_to_yaml (str): path like input

    Raises:
        ValueError: if yaml file is empty or there's an issue creating the ConfigBox
        yaml.YAMLError: if there's an issue parsing the YAML
        Exception: for any other errors

    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml, 'r') as yaml_file:
            content = yaml.safe_load(yaml_file)
            logging.info(f"yaml file: {path_to_yaml} loaded successfully")
            logging.info(f"Content of {path_to_yaml}: {content}")
            
            if not content:
                raise ValueError("YAML file is empty")
            
            return ConfigBox(content)
    except ValueError as ve:
        logging.error(f"Value error: {ve}")
        raise
    except yaml.YAMLError as ye:
        logging.error(f"Error parsing YAML file: {ye}")
        raise
    except Exception as e:
        logging.error(f"Unexpected error reading file: {e}")
        raise

In [19]:

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            if not all_schema:
                logging.warning("Schema is empty. Skipping validation.")
                return True

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation status: {validation_status}")
                else: 
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise e

In [20]:
try: 
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e 

[2025-01-14 16:17:35,886: INFO: 85844529: yaml file: config\config.yaml loaded successfully]
[2025-01-14 16:17:35,918: INFO: 85844529: Content of config\config.yaml: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/WisdomAnalyst/FIFA-21-DATA-CLEANING-/raw/refs/heads/main/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/winequality-red.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}}]
[2025-01-14 16:17:36,168: INFO: 85844529: yaml file: params.yaml loaded successfully]
[2025-01-14 16:17:36,168: INFO: 85844529: Content of params.yaml: {'key': 'val'}]
[2025-01-14 16:17:36,185: INFO: 85844529: yaml file: schema.yaml loaded successfully]
[2025-01-14 16:17:36,185: INFO: 85844529: Content of schema.yaml: {'Columns': {'fixed acidity